PP Seidel là một sự cải tiến của phương pháp lặp đơn? còn pp Gauss Seidel là một sự cải tiến của pp lặp Jacobi??

In [122]:
import numpy as np
import math

# 1. Seidel (giống lặp đơn)

Mục đích: tranh thủ tối đa lượng thông tin đã có trong quá trình tính.

1. Chuẩn hàng: 
- Công thức sai số hậu nghiệm: 
\begin{align}
\Vert x^* - x^{(k)} \Vert_{\infty} \leq \frac{\lambda}{1 - \lambda}\Vert x^{(k)} - x^{(k-1)} \Vert_{\infty}
\end{align}
với 
\begin{align}
\lambda = \max_i \frac{p_i}{1 - q_i}, \quad p_i = \sum_{j=i}^{n}|\alpha_{ij}|, \quad q_i = \sum_{j=1}^{i-1}|\alpha_{ij}|
\end{align}
- Công thức sai số tiên nghiệm: 
\begin{align}
\Vert x^{(k)} - x^* \Vert_{\infty} \leq \frac{\Vert \alpha \Vert^k}{1 - \Vert \alpha \Vert} \Vert x^{(1)} - x^{(0)} \Vert_{\infty}
\end{align}

### 1.1. Hàm tính chuẩn hàng của ma trận.

In [3]:
def rowMatrixNorm(A):
    n = len(A)
    maxRowSum = 0.0
    
    for j in range(n): 
        maxRowSum += abs(A[0][j]) # tính tổng trị tuyệt đối của hàng 1, và tạm thời gán nó là max_row. 
        
    for i in range(1, n): 
        sum = 0 
        for j in range(n): 
            sum += abs(A[i][j])
            
        if sum > maxRowSum: 
            maxRowSum = sum
            
    return maxRowSum

### 1.2. Hàm tính chuẩn cột của ma trận. 

In [4]:
def colMatrixNorm(A):
    n = len(A)
    maxColSum = 0.0
    
    for i in range(n): 
        maxColSum += abs(A[i][0]) # tính tổng trị tuyệt đối của hàng 1, và tạm thời gán nó là max_row. 
        
    for j in range(1, n): 
        sum = 0 
        for i in range(n): 
            sum += abs(A[i][j])
            
        if sum > maxColSum: 
            maxColSum = sum
            
    return maxColSum

### 1.3. Hàm tính chuẩn cực đại của véc tơ (chuẩn hàng)

In [11]:
def maxVectorNorm(x): 
    n = len(x)
    max = abs(x[0])
    
    for i in range(1, n): 
        if abs(x[i]) > max: 
            max = abs(x[i])
            
    return max

### 1.4. Hàm tính chuẩn tuyệt đối của véc tơ (chuẩn cột)

In [12]:
def absVectorNorm(x): 
    norm = 0 
    
    for i in range(len(x)): 
        norm += abs(x[i])
        
    return norm 

### 1.5. Lặp Seidel theo chuẩn hàng

In [7]:
def rowSeidel(alpha, beta): 
    n = len(alpha)
    
    x0 = np.zeros(n)
    x = np.zeros(n)
    
    for i in range(n): 
        x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
        
    print('Nhập vào sai số chung cho cả hai phép lặp:')
    error = float(input())
    
    print('Bạn muốn lặp chuẩn hàng theo công thức sai số tiên nghiệm hay hậu nghiệm: ')
    print('Tiên nghiệm: Ấn 1.')
    print('Hậu nghiệm: Ấn 2.')
    choice = int(input())
        
    
    # Chú ý: Với ma trận alpha^(1), với mỗi hàng, ta chỉ cần nhân đến phần tử trước phần tử đường chéo chính thôi 
    # chứ không cần nhân hết, vì các phần tử đường chéo chính trở đi đều bằng 0. 
    # Tiếp đến, với ma trận alpha^(2), với mỗi hàng, ta nhân từ phần tử đường chéo chính trở đi. 
    
    if choice == 1: # Lặp tiên nghiệm.
        print('k = %d: ' % (0), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x0[i]), end=' ')
        print()
        
        print('k = %d: ' % (1), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x[i]), end=' ')
        print()
        
        alNorm = rowMatrixNorm(alpha)
        print('Chuẩn hàng của alpha = %f' % (alNorm))
        k = math.ceil(math.log(error * (1 - alNorm) / maxVectorNorm(x - x0), alNorm))
        print('k = %d' % (k))
        
        for num in range(1, k): 
            print('k = %d: ' % (num + 1), end=' ')
            for i in range(n): 
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
            
    elif choice == 2: # Lặp hậu nghiệm. 
        print('k = %d: ' % (0), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x0[i]), end=' ')
        print()
        
        print('k = %d: ' % (1), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x[i]), end=' ')
        print()        
        
        k = 1
        
        # Tính lamda. 
        lamda = 0 
        for i in range(n): 
            pi = 0 
            for j in range(i, n): 
                pi += abs(alpha[i][j])
            
            qi = 0 
            for j in range(i): 
                qi += abs(alpha[i][j])
        
            if lamda < pi / (1 - qi): 
                lamda = pi / (1 - qi)
        
        print('lamda = %f' % (lamda))
        
        while lamda * maxVectorNorm(x - x0) / (1 - lamda) >= error: 
            x0 = np.copy(x)  # ndarray trong numpy hoạt động giống hệt list trong Python, nên không gán bình thường được.
            
            print('k = %d: ' % (k + 1), end=' ')
            for i in range(n):
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
            k += 1
            
    return x, k 

In [88]:
alpha = np.array([[0, -0.06, 0.02], 
                  [-0.03, 0, 0.05], 
                  [-0.01, 0.02, 0]])

beta = np.array([2, 3, 5])

In [89]:
rowSeidel(alpha, beta)

Nhập vào sai số chung cho cả hai phép lặp:
1e-3
Bạn muốn lặp chuẩn hàng theo công thức sai số tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 2.000000 x[1] = 2.940000 x[2] = 5.038800 
Chuẩn hàng của alpha = 0.080000
k = 4
k = 2:  x[0] = 1.924376 x[1] = 3.194209 x[2] = 5.044640 
k = 3:  x[0] = 1.909240 x[1] = 3.194955 x[2] = 5.044807 
k = 4:  x[0] = 1.909199 x[1] = 3.194964 x[2] = 5.044807 


(array([1.90919885, 3.19496437, 5.0448073 ]), 4)

In [92]:
rowSeidel(alpha, beta)

Nhập vào sai số chung cho cả hai phép lặp:
1e-3
Bạn muốn lặp chuẩn hàng theo công thức sai số tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
2
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 2.000000 x[1] = 2.940000 x[2] = 5.038800 
lamda = 0.080000
k = 2:  x[0] = 1.924376 x[1] = 3.194209 x[2] = 5.044640 
k = 3:  x[0] = 1.909240 x[1] = 3.194955 x[2] = 5.044807 
k = 4:  x[0] = 1.909199 x[1] = 3.194964 x[2] = 5.044807 


(array([1.90919885, 3.19496437, 5.0448073 ]), 4)

### 1.6. Lặp Seidel theo chuẩn cột.

- Công thức đánh giá sai số tiên nghiệm: 
\begin{align}
\Vert x^{(k)} - x^* \Vert_1 \leq \frac{\theta^k}{(1 - S)(1 - \theta)} \Vert x^{(1)} - x^{(0)} \Vert_1 
\end{align}
với:
\begin{align}
S = \max_j \sum_{i=j+1}^{n}|\alpha_{ij}|, \quad \theta = \max_j \frac{\sum_{i=1}^j|\alpha_{ij}|}{1 - \sum_{i=j+1}^{n}|\alpha_{ij}|}
\end{align}

### 1.7. Hàm tính S trong công thức sai số tiên nghiệm:

In [125]:
def SCalc(alpha): 
    n = len(alpha)
    
    S = 0
    for j in range(n): 
        sum = 0 
        for i in range(j + 1, n): 
            sum += abs(alpha[i][j])
            
        if S < sum: 
            S = sum
    
    return S

### 1.8. Hàm tính theta trong công thức sai số tiên nghiệm: 

In [126]:
def thetaCalc(alpha): 
    n = len(alpha)
    
    theta = 0
    for j in range(n): 
        sum1, sum2 = 0.0, 0.0 
        for i in range(j + 1): 
            sum1 += abs(alpha[i][j])
        for i in range(j + 1, n): 
            sum2 += abs(alpha[i][j])
            
        if theta < sum1 / (1 - sum2): 
            theta = sum1 / (1 - sum2)
            
    return theta

In [130]:
def colSeidel(alpha, beta): 
    n = len(alpha)
    
    x0 = np.zeros(n)
    x = np.zeros(n)
    
    for i in range(n): 
        x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
        
    print('Nhập vào sai số chung cho cả hai phép lặp:')
    error = float(input())
    
    print('Bạn muốn lặp chuẩn hàng theo công thức sai số tiên nghiệm hay hậu nghiệm: ')
    print('Tiên nghiệm: Ấn 1.')
    print('Hậu nghiệm: Ấn 2.')
    choice = int(input())
    
    if choice == 1: # Lặp tiên nghiệm.
        print('k = %d: ' % (0), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x0[i]), end=' ')
        print()
        
        print('k = %d: ' % (1), end=' ')
        for i in range(n): 
            print('x[%d] = %f' % (i, x[i]), end=' ')
        print()
        
        S = SCalc(alpha)
        theta = thetaCalc(alpha)
        
        k = math.ceil(math.log(error * (1 - S) * (1 - theta) / absVectorNorm(x - x0), theta))
        print('k = %d' % (k))
        
        for num in range(1, k): 
            print('k = %d: ' % (num + 1), end=' ')
            for i in range(n): 
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
            
    elif choice == 2: # Lặp hậu nghiệm. 
        return 'Hiện chưa có công thức lặp hậu nghiệm.'
            
    return x, k 

Ví dụ 1:

In [131]:
alpha = np.array([[0, -0.06, 0.02], 
                  [-0.03, 0, 0.05], 
                  [-0.01, 0.02, 0]])

beta = np.array([2, 3, 5])

In [132]:
colSeidel(alpha.copy(), beta.copy())

Nhập vào sai số chung cho cả hai phép lặp:
1e-3
Bạn muốn lặp chuẩn hàng theo công thức sai số tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 2.000000 x[1] = 2.940000 x[2] = 5.038800 
k = 4
k = 2:  x[0] = 1.924376 x[1] = 3.194209 x[2] = 5.044640 
k = 3:  x[0] = 1.909240 x[1] = 3.194955 x[2] = 5.044807 
k = 4:  x[0] = 1.909199 x[1] = 3.194964 x[2] = 5.044807 


(array([1.90919885, 3.19496437, 5.0448073 ]), 4)

# 2. Gauss Seidel (giống lặp Jacobi)

## 2.1. Trường hợp ma trận chéo trội hàng ($\Vert \alpha \Vert_{\infty} < 1$)

- Công thức sai số tiên nghiệm: 
\begin{align}
\Vert x^* - x^{(k)} \Vert_{\infty} \leq \frac{\lambda^{k}}{1 - \lambda}\Vert x^{(1)} - x^{(0)} \Vert_{\infty}
\end{align}
- Công thức sai số hậu nghiệm: 
\begin{align}
\Vert x^{(k)} - x^* \Vert_{\infty} \leq \frac{\lambda}{1 - \lambda} \Vert x^{(k)} - x^{(k-1)} \Vert_{\infty}
\end{align}
với: 
\begin{align}
\lambda = \max_i \frac{p_i}{1 - q_i}, \quad p_i = \sum_{j=i}^{n}|\alpha_{ij}|, \quad q_i = \sum_{j=0}^{i-1}|\alpha_{ij}|
\end{align}

In [27]:
def rowGaussSeidel(A, b): 
    n = len(A)
    
    for i in range(n): 
        temp = A[i][i]
        A[i] = A[i] / temp
        b[i] = b[i] / temp
        
    print('TA = ', A)
    print('Tb = ', b)
    print()
        
    alpha = np.identity(n) - A   # alpha lúc này trở thành ma trận chéo trội hàng. 
    beta = b
    
    x0 = np.zeros(n)
    x = np.zeros(n)
    for i in range(n): 
        x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
    
    # Áp dụng trực tiếp các công thức sai số của lặp đơn. 
    print('Bạn muốn lặp chéo trội hàng bằng công thức tiên nghiệm hay hậu nghiệm: ')
    print('Tiên nghiệm: Ấn 1.')
    print('Hậu nghiệm: Ấn 2.')
    choice = int(input())
    
    print('Nhập vào sai số: ')
    error = float(input())
    
    # Tính lamda. 
    lamda = 0 
    for i in range(n): 
        pi = 0 
        for j in range(i, n): 
            pi += abs(alpha[i][j])
            
        qi = 0 
        for j in range(i): 
            qi += abs(alpha[i][j])
        
        if lamda < pi / (1 - qi): 
            lamda = pi / (1 - qi)
        
    print('lamda = %f' % (lamda))
    
    # In ra màn hình hai kết quả lặp đầu tiên.
    print('k = %d: ' % (0), end=' ')
    for i in range(n): 
        print('x[%d] = %f' % (i, x0[i]), end=' ')
    print()
        
    print('k = %d: ' % (1), end=' ')
    for i in range(n): 
        print('x[%d] = %f' % (i, x[i]), end=' ')
    print()     
    
    if choice == 1: # Lặp tiên nghiệm.
        k = math.ceil(math.log(error * (1 - lamda) / maxVectorNorm(x - x0), lamda))
        print('k = %d' % (k))
        
        for num in range(1, k): 
            print('k = %d: ' % (num + 1), end=' ')
            for i in range(n): 
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
        
    elif choice == 2: # Lặp hậu nghiệm.
        k = 1 
        while lamda * maxVectorNorm(x - x0) / (1 - lamda) >= error:
            x0 = np.copy(x)  # ndarray trong numpy hoạt động giống hệt list trong Python, nên không gán bình thường được.
            
            print('k = %d: ' % (k + 1), end=' ')
            for i in range(n):
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
            k += 1
    
    return x, k

Ví dụ 1: 

In [28]:
# ma trận chéo trội hàng (không chéo trội cột).
A3 = np.array([[15, -5, 2], 
               [10, 35, -18], 
               [8, 4, 15]], dtype='float')

b3 = np.array([7, 12, 5], dtype='float')

In [29]:
rowGaussSeidel(A3.copy(), b3.copy())  # Lặp trội hàng, tiên nghiệm, nhanh hơn so với Jacobi cùng loại 

TA =  [[ 1.         -0.33333333  0.13333333]
 [ 0.28571429  1.         -0.51428571]
 [ 0.53333333  0.26666667  1.        ]]
Tb =  [0.46666667 0.34285714 0.33333333]

Bạn muốn lặp chéo trội hàng bằng công thức tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
Nhập vào sai số: 
1e-3
lamda = 0.720000
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 0.466667 x[1] = 0.209524 x[2] = 0.028571 
k = 23
k = 2:  x[0] = 0.532698 x[1] = 0.205351 x[2] = -0.005533 
k = 3:  x[0] = 0.535855 x[1] = 0.186910 x[2] = -0.002299 
k = 4:  x[0] = 0.529277 x[1] = 0.190453 x[2] = 0.000265 
k = 5:  x[0] = 0.530116 x[1] = 0.191532 x[2] = -0.000470 
k = 6:  x[0] = 0.530573 x[1] = 0.191023 x[2] = -0.000579 
k = 7:  x[0] = 0.530418 x[1] = 0.191012 x[2] = -0.000493 
k = 8:  x[0] = 0.530403 x[1] = 0.191060 x[2] = -0.000498 
k = 9:  x[0] = 0.530420 x[1] = 0.191053 x[2] = -0.000505 
k = 10:  x[0] = 0.530418 x[1] = 0.191050 x[2] = -0.000503 
k = 11:  x[0] = 0.530417 x[1] = 0.191051

(array([ 5.30417295e-01,  1.91050779e-01, -5.02765209e-04]), 23)

In [30]:
rowGaussSeidel(A3.copy(), b3.copy())  # Lặp trội hàng, hậu nghiệm, nhanh hơn so với Jacobi cùng loại.

TA =  [[ 1.         -0.33333333  0.13333333]
 [ 0.28571429  1.         -0.51428571]
 [ 0.53333333  0.26666667  1.        ]]
Tb =  [0.46666667 0.34285714 0.33333333]

Bạn muốn lặp chéo trội hàng bằng công thức tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
2
Nhập vào sai số: 
1e-3
lamda = 0.720000
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 0.466667 x[1] = 0.209524 x[2] = 0.028571 
k = 2:  x[0] = 0.532698 x[1] = 0.205351 x[2] = -0.005533 
k = 3:  x[0] = 0.535855 x[1] = 0.186910 x[2] = -0.002299 
k = 4:  x[0] = 0.529277 x[1] = 0.190453 x[2] = 0.000265 
k = 5:  x[0] = 0.530116 x[1] = 0.191532 x[2] = -0.000470 
k = 6:  x[0] = 0.530573 x[1] = 0.191023 x[2] = -0.000579 
k = 7:  x[0] = 0.530418 x[1] = 0.191012 x[2] = -0.000493 


(array([ 5.30418128e-01,  1.91011568e-01, -4.92752953e-04]), 7)

## 2.2. Trường hợp ma trận chéo trội cột ($\Vert \alpha \Vert_{1} < 1$)

- Công thức sai số hậu nghiệm: 
\begin{align}
\Vert x^{(0)} - x^* \Vert_1 \leq \frac{\epsilon}{(1-s)(1-\epsilon)}\Vert x^{(k)} - x^{(k-1)}\Vert_1
\end{align}
với 
\begin{align}
s = \max_j \sum_{i=j+1}^{n}|\alpha_{ij}|, \quad \epsilon = \max_j \frac{\sum_{i=1}^{j}|\alpha_{ij}|}{1-\sum_{i=j+1}^{n}|\alpha_{ij}|} \leq \Vert \alpha \Vert_1 < 1
\end{align}

- Công thức sai số tiên nghiệm: 
\begin{align}
\Vert x^{(0)} - x^* \Vert_1 \leq \frac{\epsilon^k}{(1-s)(1-\epsilon)}\Vert x^{(1)} - x^{(0)}\Vert_1
\end{align}

In [135]:
A4 = np.array([[10, 1, 1], 
              [2, 10, 1], 
              [2, 2, 10]], dtype='float')

b4 = np.array([12, 13, 14], dtype='float')

In [136]:
colGaussSeidel(A4.copy(), b4.copy())

T =  [[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.1]]

S = 0.400000
theta = 0.200000

I - TA =  [[ 0.  -0.1 -0.1]
 [-0.2  0.  -0.1]
 [-0.2 -0.2  0. ]]
Tb =  [1.2 1.3 1.4]

Bạn muốn lặp theo công thức sai số hậu nghiệm hoặc tiên nghiệm:
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
Nhập vào sai số: 
1e-3
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 1.200000 x[1] = 1.060000 x[2] = 0.948000 
gtri = 0.000150

k = 2:  x[0] = 0.999200,  x[1] = 1.005360,  x[2] = 0.999088,  
k = 3:  x[0] = 0.999555,  x[1] = 1.000180,  x[2] = 1.000053,  
k = 4:  x[0] = 0.999977,  x[1] = 0.999999,  x[2] = 1.000005,  
k = 5:  x[0] = 1.000000,  x[1] = 1.000000,  x[2] = 1.000000,  
k = 6:  x[0] = 1.000000,  x[1] = 1.000000,  x[2] = 1.000000,  


(array([1.00000002, 0.99999998, 1.        ]), 6)

In [137]:
colGaussSeidel(A4.copy(), b4.copy())

T =  [[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.1]]

S = 0.400000
theta = 0.200000

I - TA =  [[ 0.  -0.1 -0.1]
 [-0.2  0.  -0.1]
 [-0.2 -0.2  0. ]]
Tb =  [1.2 1.3 1.4]

Bạn muốn lặp theo công thức sai số hậu nghiệm hoặc tiên nghiệm:
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
2
Nhập vào sai số: 
1e-3
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 1.200000 x[1] = 1.060000 x[2] = 0.948000 


(array([1.2  , 1.06 , 0.948]), 1)

In [138]:
A5 = np.array([[10, 8, 7], 
               [3, -15, 14], 
               [-5, 2, 24]], dtype='float')

b5 = np.array([2, 6, 13], dtype='float')

In [139]:
colGaussSeidel(A5.copy(), b5.copy())

T =  [[ 0.1         0.          0.        ]
 [ 0.         -0.06666667  0.        ]
 [ 0.          0.          0.04166667]]

S = 0.408333
theta = 1.633333

I - TA =  [[ 0.         -0.8        -0.7       ]
 [ 0.2         0.          0.93333333]
 [ 0.20833333 -0.08333333  0.        ]]
Tb =  [ 0.2        -0.4         0.54166667]

Bạn muốn lặp theo công thức sai số hậu nghiệm hoặc tiên nghiệm:
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
Nhập vào sai số: 
1e-3
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 0.200000 x[1] = -0.360000 x[2] = 0.613333 
gtri = -0.000319



Exception: Giá trị lấy log < 0

In [134]:
def colGaussSeidel(A, b): 
    n = len(A)
    
    T = np.identity(n, dtype='float')
    for i in range(n): 
        T[i][i] = T[i][i] / A[i][i]
        
    print('T = ', T)
    print()
    
    alpha = np.identity(n) - np.matmul(T, A)
    beta = np.matmul(T, b)
    
    S = SCalc(alpha)
    theta = thetaCalc(alpha)
    
    print('S = %f' % (S))
    print('theta = %f' % (theta))
    print()
    
    print('I - TA = ', alpha)
    print('Tb = ', beta)
    print()
    
    # Ta sẽ áp dụng công thức sai số trực tiếp lên x. 
    print('Bạn muốn lặp theo công thức sai số hậu nghiệm hoặc tiên nghiệm:')
    print('Tiên nghiệm: Ấn 1.')
    print('Hậu nghiệm: Ấn 2.')
    choice = int(input())
    
    print('Nhập vào sai số: ')
    error = float(input())
    
    x0 = np.zeros(n)
    x = np.zeros(n)
    for i in range(n): 
        x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
    
    # In ra màn hình hai kết quả lặp đầu tiên.
    print('k = %d: ' % (0), end=' ')
    for i in range(n): 
        print('x[%d] = %f' % (i, x0[i]), end=' ')
    print()
        
    print('k = %d: ' % (1), end=' ')
    for i in range(n): 
        print('x[%d] = %f' % (i, x[i]), end=' ')
    print()     
    
    if choice == 1:  # Lặp tiên nghiệm
        gtri = error * (1 - S) * (1 - theta) / absVectorNorm(x - x0)
        print('gtri = %f' % (gtri))
        print()
        if gtri < 0: 
            raise Exception('Giá trị lấy log < 0')
        
        k = math.ceil(math.log(error * (1 - S) * (1 - theta) / absVectorNorm(x - x0), theta))
        
        for num in range(1, k):
            print('k = %d: ' % (num + 1), end=' ')
            for i in range(n):
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f, ' % (i, x[i]), end=' ')
            print()
        
    elif choice == 2: # Lặp hậu nghiệm.
        k = 1 
        while error * (1 - S) * (1 - theta) / (absVectorNorm(x - x0) * theta) >= error: 
            x0 = np.copy(x)  # ndarray trong numpy hoạt động giống hệt list trong Python, nên không gán bình thường được.
            
            print('k = %d: ' % (k + 1), end=' ')
            for i in range(n):
                x[i] = np.dot(alpha[i][:i], x[:i]) + np.dot(alpha[i][i:], x[i:]) + beta[i] 
                print('x[%d] = %f' % (i, x[i]), end=' ')
            print()
            k += 1
    
    return x, k

## 2.3. Hàm kiểm tra tính chéo trội hàng của ma trận.

In [104]:
def rowDominant(A): 
    n = len(A)
    
    # kiểm tra chéo trội hàng. 
    for i in range(n): 
        sum = 0 
        for j in range(n): 
            if j != i: 
                sum += abs(A[i][j])
        
        if abs(A[i][i]) <= sum: 
            return False 
        
    return True 

## 2.4. Hàm kiểm tra tính chéo trội cột của ma trận.

In [105]:
def colDominant(A): 
    n = len(A)
    
    for j in range(n): 
        sum = 0
        for i in range(n): 
            if i != j: 
                sum += abs(A[i][j])
                
        if abs(A[j][j]) <= sum: 
            return False 
        
    return True 

## 2.5. Hàm thực thi chính 

In [106]:
def mainGaussSeidel(A, b): 
    if rowDominant(A): 
        print('Ma trận là chéo trội hàng.')
        x, k = rowGaussSeidel(A, b)
    elif colDominant(A): 
        print('Ma trận là chéo trội cột.')
        x, k = colGaussSeidel(A, b)
    else: 
        return 'Ma trận A không chéo trội.'
    
    return x, k

In [109]:
A4 = np.array([[10, 1, 1], 
              [2, 10, 1], 
              [2, 2, 10]], dtype='float')

b4 = np.array([12, 13, 14], dtype='float')

In [110]:
mainGaussSeidel(A4.copy(), b4.copy())

Ma trận là chéo trội hàng.
TA =  [[1.  0.1 0.1]
 [0.2 1.  0.1]
 [0.2 0.2 1. ]]
Tb =  [1.2 1.3 1.4]

Bạn muốn lặp chéo trội hàng bằng công thức tiên nghiệm hay hậu nghiệm: 
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
1
Nhập vào sai số: 
1e-3
lamda = 0.200000
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 1.200000 x[1] = 1.060000 x[2] = 0.948000 
k = 5
k = 2:  x[0] = 0.999200 x[1] = 1.005360 x[2] = 0.999088 
k = 3:  x[0] = 0.999555 x[1] = 1.000180 x[2] = 1.000053 
k = 4:  x[0] = 0.999977 x[1] = 0.999999 x[2] = 1.000005 
k = 5:  x[0] = 1.000000 x[1] = 1.000000 x[2] = 1.000000 


(array([0.99999958, 0.9999996 , 1.00000016]), 5)

In [74]:
mainGaussSeidel(A4.copy(), b4.copy())

Ma trận là chéo trội cột.
T =  [[ 0.1         0.          0.        ]
 [ 0.         -0.06666667  0.        ]
 [ 0.          0.          0.04166667]]

S = 0.408333
theta = 1.633333

TA =  [[ 10.   8.   7.]
 [  3. -15.  14.]
 [ -5.   2.  24.]]
Tb =  [ 2.  6. 13.]

Bạn muốn lặp theo công thức sai số hậu nghiệm hoặc tiên nghiệm:
Tiên nghiệm: Ấn 1.
Hậu nghiệm: Ấn 2.
2
Nhập vào sai số: 
1e-3
k = 0:  x[0] = 0.000000 x[1] = 0.000000 x[2] = 0.000000 
k = 1:  x[0] = 0.200000 x[1] = -0.360000 x[2] = 0.613333 


(array([ 0.2       , -0.36      ,  0.61333333]), 1)